## Data preparation
This notebook is a sequel to the preliminary data exploration notebook. This notebook will develop a key derived feature (borrower zip) and then break up the dataset into training and test datasets. Note that the feature developed is based on the observations in the _training dataset_. Outlier analysis is not done. There are some big loans, these could be removed from the data set. A standard practice is to fence the target values from the median at a distance of 1.5 times the inter quartile range. THis will be done subsequently. Elkan's paper develops a theory for the proportion by which the majority class should be reduced to create a balanced dataset on the basis of the false positve and the false negative costs. In this work, Hart's condensed nearest neighbor method was used to resample the dataset. The cost ratio is calculated for illustrative purposes. As will be seen, the threshold $p^*$ from this calculation is pretty close to $0.5$.  

### Elkans Undersampling Theory calculation
The reduction in majority class based on cost basis is shown, but not used in this work

In [1]:
# neg (good_creditor) = 0, positive (bad_creditor) = 1
c_10 = 169040.98 # false_positive_cost 
c_01 = 199617.34 # false_negative_ cost
cost_ratio = c_01/c_10

In [2]:
negative_undersampling = c_10/c_01

In [3]:
# under sample the negative (majority class) by 84%
negative_undersampling

0.8468251305222283

In [4]:
import pandas as pd
fp = "../data/sba_real_estate.csv"
df = pd.read_csv(fp)

In [5]:
df["LoanStatus"].value_counts()

LoanStatus
PIF       4543
CANCLD    1444
CHGOFF     264
Name: count, dtype: int64

In [6]:
df.columns

Index(['BorrName', 'BorrZip', 'BankName', 'BankZip', 'GrossApproval',
       'SBAGuaranteedApproval', 'InitialInterestRate', 'TermInMonths',
       'ProjectState', 'SBADistrictOffice', 'CongressionalDistrict',
       'BusinessType', 'RevolverStatus', 'JobsSupported', 'LoanStatus'],
      dtype='object')

In [7]:
cols = df.columns.tolist()
num_cols = ["JobsSupported", "InitialInterestRate", "TermInMonths", "GrossApproval", "SBAGuaranteedApproval"]
target_cols = ["LoanStatus"]
cat_cols = [ c for c in cols if c not in(target_cols + num_cols)]

## Estimate the cardinality of the categorical columns

In [8]:
df_cat = df[cat_cols]
df_cat = df_cat.astype(str)

In [9]:
cardinality = {}
for c in cat_cols:
    num_vals = len(df[c].unique())
    cardinality[c] = num_vals
    

In [10]:
cardinality

{'BorrName': 5409,
 'BorrZip': 3837,
 'BankName': 895,
 'BankZip': 841,
 'ProjectState': 55,
 'SBADistrictOffice': 74,
 'CongressionalDistrict': 54,
 'BusinessType': 3,
 'RevolverStatus': 2}

In [11]:
df_nf = df[num_cols]
df_nf = df_nf.astype(float)

## Use masking to split dataset into train and test

In [12]:
import numpy as np
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_test = df[~msk]

## Develop a feature for bad borrower zip code membership

In [13]:
bz = df_train[df_train["LoanStatus"].isin(["CANCLD", "CHGOFF"])]["BorrZip"]
gz = df_train[df_train["LoanStatus"].isin(["PIF"])]["BorrZip"]
bzs = set(bz.values)
gzs = set(gz.values)
xg = gzs - bzs
xb = bzs - gzs

def bad_borr_zip(zip):
    if zip in xg:
        return "D"
    else:
        return "ND"
df_train["D_BorrZip"] = df_train["BorrZip"].apply(bad_borr_zip)
df_test["D_BorrZip"] = df_test["BorrZip"].apply(bad_borr_zip)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_9688/3281216051.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["D_BorrZip"] = df_train["BorrZip"].apply(bad_borr_zip)
/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_9688/3281216051.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["D_BorrZip"] = df_test["BorrZip"].apply(bad_borr_zip)


## Develop a feature for bad bank zip membership

In [14]:
bz = df_train[df_train["LoanStatus"].isin(["CANCLD", "CHGOFF"])]["BankZip"]
gz = df_train[df_train["LoanStatus"].isin(["PIF"])]["BankZip"]
bzs = set(bz.values)
gzs = set(gz.values)
xg = gzs - bzs
xb = bzs - gzs

def bad_bank_zip(zip):
    if zip in xg:
        return "D"
    else:
        return "ND"
df_train["D_BankZip"] = df_train["BankZip"].apply(bad_bank_zip)
df_test["D_BankZip"] = df_test["BankZip"].apply(bad_bank_zip)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_9688/3228291768.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["D_BankZip"] = df_train["BankZip"].apply(bad_bank_zip)
/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_9688/3228291768.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["D_BankZip"] = df_test["BankZip"].apply(bad_bank_zip)


## Write test and train datasets to disk

In [15]:
pred_cols = num_cols + ["D_BankZip", "D_BorrZip"]
keep_cols = pred_cols + ["LoanStatus"]
df_train = df_train[keep_cols]
df_test = df_test[keep_cols]

In [16]:
df_train = pd.concat([pd.get_dummies(df_train[pred_cols], drop_first=True, dtype=float),
           df_train["LoanStatus"]], axis=1)
df_train["LoanStatus"] = df_train["LoanStatus"].replace({"PIF": 0, "CANCLD":1, "CHGOFF": 1})

In [17]:
df_test = pd.concat([pd.get_dummies(df_test[pred_cols], drop_first=True, dtype=float),
           df_test["LoanStatus"]], axis=1)
df_test["LoanStatus"] = df_test["LoanStatus"].replace({"PIF": 0, "CANCLD":1, "CHGOFF": 1})

In [18]:
df_train

,JobsSupported,InitialInterestRate,TermInMonths,GrossApproval,SBAGuaranteedApproval,D_BankZip_ND,D_BorrZip_ND,LoanStatus
0,35,5.26,84,250000.0,125000.0,1.0,0.0,0
1,1,6.00,240,150000.0,135000.0,0.0,0.0,0
2,0,6.00,309,1350000.0,1215000.0,1.0,1.0,1
3,4,6.00,300,900000.0,810000.0,1.0,1.0,0
4,4,6.00,120,504000.0,453600.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
6244,9,5.90,120,382700.0,287025.0,1.0,1.0,1
6245,21,6.00,120,1050000.0,787500.0,1.0,1.0,1
6246,25,3.50,120,1000000.0,750000.0,1.0,1.0,1
6247,7,6.00,120,200000.0,150000.0,1.0,1.0,1


In [19]:
fp_train = "../data/sba_training_dataset.csv"
df_train.to_csv(fp_train, index=False)
fp_test = "../data/sba_test_dataset.csv"
df_test.to_csv(fp_test, index=False)

In [20]:
df_train

,JobsSupported,InitialInterestRate,TermInMonths,GrossApproval,SBAGuaranteedApproval,D_BankZip_ND,D_BorrZip_ND,LoanStatus
0,35,5.26,84,250000.0,125000.0,1.0,0.0,0
1,1,6.00,240,150000.0,135000.0,0.0,0.0,0
2,0,6.00,309,1350000.0,1215000.0,1.0,1.0,1
3,4,6.00,300,900000.0,810000.0,1.0,1.0,0
4,4,6.00,120,504000.0,453600.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
6244,9,5.90,120,382700.0,287025.0,1.0,1.0,1
6245,21,6.00,120,1050000.0,787500.0,1.0,1.0,1
6246,25,3.50,120,1000000.0,750000.0,1.0,1.0,1
6247,7,6.00,120,200000.0,150000.0,1.0,1.0,1


In [21]:
df_train["LoanStatus"].value_counts()

LoanStatus
0    3623
1    1338
Name: count, dtype: int64

In [22]:
df_test["LoanStatus"].value_counts()

LoanStatus
0    920
1    370
Name: count, dtype: int64